In [1]:
import sympy

In [2]:
a_Ax, a_Ay, a_Bx, a_By = sympy.symbols("a_A^1 a_A^2 a_B^1 a_B^2")
a_eles = [a_Ax, a_Ay, a_Bx, a_By]
a = sympy.Matrix(a_eles)
display(a)
m_A, m_B = sympy.symbols("m_A m_B")
M = sympy.diag(*[m_A, m_A, m_B, m_B])
display(M)
F_E, g = sympy.symbols("F_E g")
F_E = sympy.Matrix([0, -g, 0, -g])
display(F_E)
F_C_Ax, F_C_Ay, F_C_Bx, F_C_By = sympy.symbols("F_A^1 F_A^2 F_B^1 F_B^2")
F_C_eles = [F_C_Ax, F_C_Ay, F_C_Bx, F_C_By]
F_C = sympy.Matrix(F_C_eles)
display(F_C)

Matrix([
[a_A^1],
[a_A^2],
[a_B^1],
[a_B^2]])

Matrix([
[m_A,   0,   0,   0],
[  0, m_A,   0,   0],
[  0,   0, m_B,   0],
[  0,   0,   0, m_B]])

Matrix([
[ 0],
[-g],
[ 0],
[-g]])

Matrix([
[F_A^1],
[F_A^2],
[F_B^1],
[F_B^2]])

In [3]:
x_A, y_A, x_B, y_B = sympy.symbols("x_A y_A x_B y_B")
C = sympy.Matrix([x_A, y_A, (x_B-x_A)*(x_B-x_A) + (y_B-y_A)*(y_B-y_A)])
display(C)

lam_eles = sympy.symbols(f"\lambda_1:{C.shape[0]+1}")
lam = sympy.Matrix(lam_eles)
display(lam)

Matrix([
[                              x_A],
[                              y_A],
[(-x_A + x_B)**2 + (-y_A + y_B)**2]])

Matrix([
[\lambda_1],
[\lambda_2],
[\lambda_3]])

In [4]:
q = sympy.Matrix([x_A, y_A, x_B, y_B])
J = C.jacobian(q)

J

Matrix([
[            1,             0,              0,              0],
[            0,             1,              0,              0],
[2*x_A - 2*x_B, 2*y_A - 2*y_B, -2*x_A + 2*x_B, -2*y_A + 2*y_B]])

In [5]:
print(sympy.latex(J))

\left[\begin{matrix}1 & 0 & 0 & 0\\0 & 1 & 0 & 0\\2 x_{A} - 2 x_{B} & 2 y_{A} - 2 y_{B} & - 2 x_{A} + 2 x_{B} & - 2 y_{A} + 2 y_{B}\end{matrix}\right]


In [6]:
v_Ax, v_Ay, v_Bx, v_By = sympy.symbols("v_A^1 v_A^2 v_B^1 v_B^2")
dd_C = sympy.Matrix([
	a_Ax,
    a_Ay,
	2*(
		(v_Bx - v_Ax) * (v_Bx - v_Ax) +
		(v_By - v_Ay) * (v_By - v_Ay) +
		(x_B - x_A) * (a_Bx - a_Ax) + 
		(y_B - y_A) * (a_By - a_Ay)
	)
])
dd_C

Matrix([
[                                                                                                            a_A^1],
[                                                                                                            a_A^2],
[2*(-a_A^1 + a_B^1)*(-x_A + x_B) + 2*(-a_A^2 + a_B^2)*(-y_A + y_B) + 2*(-v_A^1 + v_B^1)**2 + 2*(-v_A^2 + v_B^2)**2]])

In [7]:
cond1 = sympy.Eq(dd_C, sympy.Matrix([0]*dd_C.shape[0]))
cond2 = sympy.Eq(F_C, sympy.Transpose(J) * lam)
motion_equ = sympy.Eq(F_C + F_E, M * a)

system = [cond1, cond2, motion_equ]
for equ in system:
	display(equ)

Eq(Matrix([
[                                                                                                            a_A^1],
[                                                                                                            a_A^2],
[2*(-a_A^1 + a_B^1)*(-x_A + x_B) + 2*(-a_A^2 + a_B^2)*(-y_A + y_B) + 2*(-v_A^1 + v_B^1)**2 + 2*(-v_A^2 + v_B^2)**2]]), Matrix([
[0],
[0],
[0]]))

Eq(Matrix([
[F_A^1],
[F_A^2],
[F_B^1],
[F_B^2]]), Matrix([
[\lambda_1 + \lambda_3*(2*x_A - 2*x_B)],
[\lambda_2 + \lambda_3*(2*y_A - 2*y_B)],
[           \lambda_3*(-2*x_A + 2*x_B)],
[           \lambda_3*(-2*y_A + 2*y_B)]]))

Eq(Matrix([
[    F_A^1],
[F_A^2 - g],
[    F_B^1],
[F_B^2 - g]]), Matrix([
[a_A^1*m_A],
[a_A^2*m_A],
[a_B^1*m_B],
[a_B^2*m_B]]))

In [8]:
import sympy.solvers
unknowns = [*lam_eles, *F_C_eles, *a_eles]
res = sympy.solvers.solve(system, *unknowns)

In [9]:
for item in unknowns:
	value = res[item]
	value_pair = [
		(x_A, 0),
		(y_A, 0),
		(x_B, 1),
		(y_B, -1),
		(v_Ax, 0),
		(v_Ay, 0),
		(v_Bx, 0),
		(v_By, 0),
		(m_A, 1),
		(m_B, 1),
	]
	for p in value_pair:
		value = value.subs(*p)
	display(sympy.Eq(item, value, evaluate=False))

Eq(\lambda_1, -g/2)

Eq(\lambda_2, 3*g/2)

Eq(\lambda_3, -g/4)

Eq(F_A^1, 0)

Eq(F_A^2, g)

...

Eq(F_B^2, g/2)

Eq(a_A^1, 0)

Eq(a_A^2, 0)

Eq(a_B^1, -g/2)

Eq(a_B^2, -g/2)